<a href="https://colab.research.google.com/github/RaulLima2/-Algoritmos-e-Estrutura-de-Dado/blob/master/Predi%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparação de Ambiente

In [1]:
import numpy as np
import pandas as pd
import requests
# pre-processamento
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
# metricas de avaliação
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
# modelos
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
# plotar grafico
import seaborn as sns
import matplotlib.pyplot as plt

## Carregamento dos dados

In [2]:
req = requests.get('https://raw.githubusercontent.com/coinmetrics/data/master/csv/btc.csv')
url_content = req.content
csv_file = open('dataset.csv', 'wb')
csv_file.write(url_content)
csv_file.close()

In [3]:
dataset:pd.DataFrame = pd.read_csv('dataset.csv')

<ipython-input-3-ea087921371a>:1: DtypeWarning: Columns (146) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset:pd.DataFrame = pd.read_csv('dataset.csv')


## Pre-Processamento

In [4]:
dataset.head(10)

,time,AdrActCnt,AdrBal1in100KCnt,AdrBal1in100MCnt,AdrBal1in10BCnt,AdrBal1in10KCnt,AdrBal1in10MCnt,AdrBal1in1BCnt,AdrBal1in1KCnt,AdrBal1in1MCnt,...,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,VelCur1yr,VtyDayRet180d,VtyDayRet30d,principal_market_price_usd,principal_market_usd
0,2009-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-01-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2009-01-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2009-01-09,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,...,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
7,2009-01-10,61.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,...,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
8,2009-01-11,95.0,174.0,174.0,174.0,174.0,174.0,174.0,174.0,174.0,...,NaN,10.0,NaN,10.0,NaN,0.001156,NaN,NaN,NaN,NaN
9,2009-01-12,101.0,272.0,272.0,272.0,272.0,272.0,272.0,267.0,272.0,...,NaN,4.0,NaN,1.0,NaN,0.002547,NaN,NaN,NaN,NaN


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5283 entries, 0 to 5282
Columns: 147 entries, time to principal_market_usd
dtypes: float64(145), object(2)
memory usage: 5.9+ MB


In [6]:
dataset.describe()

,AdrActCnt,AdrBal1in100KCnt,AdrBal1in100MCnt,AdrBal1in10BCnt,AdrBal1in10KCnt,AdrBal1in10MCnt,AdrBal1in1BCnt,AdrBal1in1KCnt,AdrBal1in1MCnt,AdrBalCnt,...,TxTfrValAdjNtv,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,VelCur1yr,VtyDayRet180d,VtyDayRet30d,principal_market_price_usd
count,5.282000e+03,5282.00000,5.282000e+03,5.282000e+03,5282.000000,5282.000000,5.282000e+03,5282.000000,5282.000000,5.282000e+03,...,5.282000e+03,4.721000e+03,5024.000000,4721.000000,5.024000e+03,4.721000e+03,5276.000000,4541.000000,4691.000000,478.000000
mean,4.492407e+05,11748.40780,1.144475e+06,6.506921e+06,905.754070,303134.184968,3.068674e+06,55.646346,87895.164332,1.515662e+07,...,2.557190e+05,2.680181e+09,78.052681,11169.635499,4.484420e+01,5.282497e+01,26.346037,0.046069,0.041959,25567.120397
std,3.856275e+05,9837.31821,9.351104e+05,6.563188e+06,677.097715,173948.075739,2.888999e+06,31.000056,26472.649807,1.545967e+07,...,1.759636e+05,4.955626e+09,614.363131,23906.236793,5.799803e+02,5.929688e+01,21.738605,0.019881,0.027260,7689.111440
min,0.000000e+00,0.00000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000e+00,4.647065e+02,0.010000,0.762873,1.000000e-08,8.203471e-08,0.000000,0.020298,0.007481,15760.140000
25%,3.612425e+04,8937.00000,1.875248e+05,3.948992e+05,871.000000,135278.000000,3.166105e+05,43.000000,80819.750000,8.029112e+05,...,1.399974e+05,3.767161e+07,1.032392,751.471597,9.019990e-03,5.068396e+00,11.902670,0.032614,0.025312,19992.907500
50%,4.640175e+05,9167.00000,1.004292e+06,3.466957e+06,937.000000,343637.000000,2.008772e+06,58.000000,99629.000000,7.736917e+06,...,2.381093e+05,3.663511e+08,2.756907,3257.584777,1.985864e-02,2.661650e+01,20.915100,0.040676,0.034607,23261.535000
75%,7.903788e+05,9401.00000,2.040891e+06,1.263110e+07,985.000000,469518.250000,5.847475e+06,66.000000,105343.500000,2.805007e+07,...,3.554229e+05,2.638205e+09,11.799593,10128.455375,5.621206e-02,8.725600e+01,36.052091,0.050956,0.049458,28677.107500
max,1.366494e+06,54047.00000,2.979589e+06,1.978637e+07,9424.000000,551479.000000,9.169064e+06,958.000000,112006.000000,4.808629e+07,...,2.090898e+06,3.646588e+10,22175.000000,506174.406752,2.217500e+04,5.348536e+02,91.444767,0.109532,0.191722,47448.430000


In [7]:
pd.unique(dataset['principal_market_usd'])

array([nan, 'coinbase-btc-usd-spot', 'ftx-btc-usd-spot',
       'ftx.us-btc-usd-spot', 'binance.us-btc-usd-spot',
       'kraken-btc-usd-spot'], dtype=object)

In [8]:
dataset.boxplot(column=dataset.columns.tolist()[1:len(dataset.columns.tolist())-1], figsize=(200,200))

<Axes: >

In [9]:
dataset.fillna(0,inplace=True)

In [10]:
column:list[int] = list(map(int,dataset['PriceUSD'].tolist()))

In [11]:
dataset.drop(columns=['PriceUSD'], inplace=True)

In [12]:
columns:list = dataset.columns.tolist()

### Observações
* Os dados não se encontram normalizados
* Têm coluna com campo do tipo texto

## Processamento

In [13]:
lista_unica = pd.unique(dataset['principal_market_usd']).tolist()
def change_text(texto:str):
  return lista_unica.index(texto) + 1

In [14]:
dataset['principal_market_usd'] = dataset['principal_market_usd'].apply(change_text)

In [15]:
dataset['time_day'] = pd.to_datetime(dataset['time'])
dataset['time_month'] = pd.to_datetime(dataset['time'])
dataset['time_year'] = pd.to_datetime(dataset['time'])

In [16]:
dataset['time_day'] = dataset['time_day'].dt.day
dataset['time_month'] = dataset['time_month'].dt.month
dataset['time_year'] = dataset['time_year'].dt.year

In [17]:
dataset.drop(columns=['time'],inplace=True)

In [18]:
columns:list[float] = dataset.columns.tolist()

In [19]:
dataset[columns] = MinMaxScaler().fit_transform(dataset[columns])

In [20]:
dataset

,AdrActCnt,AdrBal1in100KCnt,AdrBal1in100MCnt,AdrBal1in10BCnt,AdrBal1in10KCnt,AdrBal1in10MCnt,AdrBal1in1BCnt,AdrBal1in1KCnt,AdrBal1in1MCnt,AdrBalCnt,...,TxTfrValMedNtv,TxTfrValMedUSD,VelCur1yr,VtyDayRet180d,VtyDayRet30d,principal_market_price_usd,principal_market_usd,time_day,time_month,time_year
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.066667,0.000000,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.100000,0.000000,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.133333,0.000000,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.166667,0.000000,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.200000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5278,0.631982,0.166318,1.000000,0.992893,0.102716,0.999570,0.999437,0.043841,0.905005,1.000000,...,3.746110e-08,0.041180,0.220763,0.226572,0.115093,0.554910,0.2,0.533333,0.454545,1.0
5279,0.646811,0.166004,0.999469,0.992616,0.102929,0.999518,0.999434,0.043841,0.905130,0.999626,...,1.623450e-08,0.017738,0.217398,0.225948,0.115197,0.558639,0.2,0.566667,0.454545,1.0
5280,0.673660,0.166115,0.999433,0.991775,0.102929,0.999951,0.998600,0.043841,0.905371,0.998776,...,5.103495e-08,0.056660,0.215834,0.226058,0.116017,0.555054,0.2,0.600000,0.454545,1.0
5281,0.696619,0.166152,0.999851,0.992497,0.103247,1.000000,0.999336,0.043841,0.905094,0.999723,...,3.876392e-08,0.045491,0.214227,0.228900,0.126961,0.565620,0.2,0.633333,0.454545,1.0


## Preparação para Teste e Treino

In [21]:
train_x, test_x , train_y , test_y = train_test_split(dataset, column, test_size=0.8, shuffle=True)

## Treinar Modelo

In [22]:
def train_classifier(classifier, X_train, X_test, y_train, y_test):
    SEED = 512
    np.random.seed(SEED)

    cv = KFold(n_splits=5, shuffle = True)

    model = classifier()
    model.fit(X_train, y_train)

    predict = model.predict(X_test)
    print("Accuracy: %.2f%%" % (accuracy_score(y_test, predict) * 100))
    print("Precision: %.2f%%" % (precision_score(y_test, predict, average='micro') * 100))
    print("Recall: %.2f%%" % (recall_score(y_test, predict, average='micro') * 100))
    print("F1 Score: %.2f%%" % (f1_score(y_test, predict, average='micro') * 100))

    results = cross_validate(model, X_train, y_train, cv=cv)

    mean = results['test_score'].mean()
    std = results['test_score'].std()
    print("Margem de erro: [%.2f, %.2f]" % ((mean - 2 * std) * 100, (mean + 2 * std) * 100))

    matriz_confusao = confusion_matrix(y_test, predict)
    plt.figure(figsize = (5, 4))
    sns.set()
    sns.heatmap(matriz_confusao, annot= True, fmt="d").set(xlabel="Predição", ylabel="Real")

#### SVC

In [ ]:
train_classifier(SVC, train_x, test_x, train_y, test_y)

Accuracy: 19.14%
Precision: 19.14%
Recall: 19.14%
F1 Score: 19.14%
Margem de erro: [11.23, 22.87]


#### Decision Tree Classifier

In [ ]:
train_classifier(DecisionTreeClassifier,train_x, test_x, train_y, test_y)

#### KNeighBors Classifier

In [ ]:
train_classifier(KNeighborsClassifier,train_x, test_x, train_y, test_y)

#### Multi - Layer Perceptron

In [ ]:
train_classifier(MLPClassifier, train_x, test_x, train_y, test_y)

#### Random Forest Classifier

In [ ]:
train_classifier(RandomForestClassifier, train_x, test_x, train_y, test_y)

#### Bagging Classifier

In [ ]:
train_classifier(BaggingClassifier, train_x, test_x, train_y, test_y)

#### Extra Trees Classifier

In [ ]:
train_classifier(ExtraTreesClassifier, train_x, test_x, train_y, test_y)

#### GradientBoostingClassifier

In [ ]:
train_classifier(GradientBoostingClassifier, train_x, test_x, train_y, test_y)

Accuracy: 22.47%
Precision: 22.47%
Recall: 22.47%
F1 Score: 22.47%


KeyboardInterrupt: ignored

#### Hist Gradient Boosting Classifier

In [ ]:
train_classifier(HistGradientBoostingClassifier, train_x, test_x, train_y, test_y)

## Analise